In [ ]:
import requests
from pprint import pprint
from datetime import date, datetime, timedelta
import json
import numpy as np
import pandas as pd
import urllib.parse

In [ ]:
#THIS IS ME TRYING TO GET COLAB TO RUN THIS NOTEBOOK
import os
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
pip install geojson

In [ ]:
#THIS IS ME TRYING TO GET COLAB TO RUN THIS NOTEBOOK
import geojson
with open("/content/drive/My Drive/Colab Notebooks/colab project/Chatbot/CHP_CT_CENTRE_V1_20220914.geojson", encoding="utf8") as f:
    gj = geojson.load(f)

In [ ]:
def nearestvacc(userin):
    for i in gj['features']:
      if i['properties']['DISTRICT_EN'] == userin:
        return (f"Go to {i['properties']['CENTRENAME_EN']}, at {i['properties']['ADDRESS_EN']}, {i['properties']['OPENINGHOUR_EN']}")

In [ ]:
urlCaseApi = 'https://chp-dashboard.geodata.gov.hk/covid-19/data/keynum.json?_='
nowTime = datetime.now()
timeStamp = datetime.timestamp(nowTime)
urlCase = urlCaseApi + str(int(timeStamp))
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'
}
rCase = requests.get(urlCase, headers=headers)
rCaseDict = rCase.json()
casenum = rCaseDict["LocalCasesAdded"]

In [ ]:
#user input address
#address = '18 Whitfield Rd, Causeway Bay, Hong Kong' xccelerate address
def nearesthospital(address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    try:
        lat = (response[0]["lat"])
    except:
        print("Invalid Address")
    try:
        lon = (response[0]["lon"])
    except:
        print("Invalid Address")
    #nearest hospitals within 2km
    latmax = str(float(lat)+0.018)
    latmin = str(float(lat)-0.018)
    lonmax = str(float(lon)+0.018)
    lonmin = str(float(lon)-0.018)
    baseurl = "https://geodata.gov.hk/gs/api/v1.0.0/geoDataQuery?q=%7Bv%3A%221%2E0%2E0%22%2Cid%3A25714788-aea1-4aaa-a76c-8a943513b5d0%2Clang%3A%22ENG%22"
    urllonmin = "%2ClongMin%3A" + lonmin
    urllatmin = "%2ClatMin%3A" + latmin
    urllonmax = "%2ClongMax%3A" + lonmax
    urllatmax = "%2ClatMax%3A" + latmax
    end = "%7D"
    urlHospital = baseurl + urllonmin + urllatmin + urllonmax + urllatmax + end
    hospitals = requests.get(urlHospital)
    hospitaldict = hospitals.json()
    if hospitaldict['features'][0]['properties']['Facility Name'] == "":
      return "No found hospital within 2km. Call 112 for closest hospital."
    else:
      return f"A hospital within 2km of your location is {hospitaldict['features'][0]['properties']['Facility Name']}, located at {hospitaldict['features'][0]['properties']['Address']}. Contact them at {hospitaldict['features'][0]['properties']['Telephone']}"

In [ ]:
import requests
import json
import pprint
import time
def get_geo(building_name):
    headers ={"Accept": "application/json", "Accept-Language" : "en"}
    url = "https://www.als.ogcio.gov.hk/lookup"
    params = {
        "q":building_name
    }
    res = requests.get(url, params=params, headers=headers) #200 means success
    d = json.loads(res.text)
    latitude = d['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Latitude']
    longitude = d['SuggestedAddress'][0]['Address']['PremisesAddress']['GeospatialInformation']['Longitude']
    return latitude, longitude

In [ ]:
def datetost(date):    
    return  (date).strftime("%m-%d-%Y")

def figure(date):
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{datetost(date)}.csv'
    df = pd.read_csv(url)
    return list(df[df["Combined_Key"].str.contains("Hong Kong")][["Confirmed","Deaths"]].iloc[0])

def case(date):
    """return confirmed and deaths cases"""
    acc_cases = np.array(figure(date))
    casesdiff = np.array(figure(date))-np.array(figure(date - timedelta(days=1)))
    return (f"""Last update: {datetost(date)}
Cases: {acc_cases[0]}  (+{casesdiff[0]})
Dealths: {acc_cases[1]}  (+{casesdiff[1]})
"+" shows new cases reported""")

def display(date=date.today()):
  i=0
  while True:
    try:
      return case(date-timedelta(days=i))
    except:
      i+=1


In [ ]:
# the json
training_set = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "What's up"],
              "responses": ["Hey!", "Hello", "How are you doing?", "Greetings!", "How do you do?", "Hi"],
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am 1 years-old", "I was born in 2021", "My birthday is September 12th and I was born in 2021", "12/09/2021"]
             },
             {"tag": "guide",
              "patterns": ["I am bored", "So bored", "Can you talk to me"], 
              "responses": ["I am available","I am free", "You could talk to me", "I am not busy"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?","what is your name?","What are you?"],
              "responses": ["My name is Sarah", "I'm Sarah", "Sarah"]
             },
              {"tag": "reply",
              "patterns": ["OK", "Thank you", "Thanks?","thx","Thank you for your help"],
              "responses": ["You are welcome", "Welcome!", "^_^"]
             },
             {"tag": "bye",
              "patterns": [ "bye", "See you", "bye bye","See you soon"],
              "responses": ["Nice speaking to you", "See you later", "See you soon"]
             },
             {"tag": "new infections",
              "patterns": ["How many covid infections today?", "how many new covid cases?", "how many covid infections", "how many more people got covid", "how many people died from covid 19"],
              "responses": [display()]
             },
             {"tag": "booking",
              "patterns": [ "set up an appointment", "book a time", "book an appointment", "schedule an appointment"],
              "responses": ["""To book an appointment, go to https://www.ha.org.hk/goto/gopc/en/ and contact a hospital for assistance. 
              If you are suffering from symptoms of Covid-19, go to https://www.ha.org.hk/goto/pre-admission_support/en/ for information about what to do."""]
             },
             {"tag": "hospital location",
              "patterns": ["Where is the nearest medical facility?", "How close is the nearest hospital?", "closest medical provider"],
              "responses": ["To find the closest hospital, type Nearest Hospital (address). Go to https://www.finddoc.com/en/practices to find the nearest doctor."]
             },
             {"tag": "function",
              "patterns": ["What can you do?", "How do you help me?", "What is your purpose","What things can you do?"],
              "responses": ["I can help you on providing some essential informations on healthiness."]
             },
             {"tag": "mild illness",
              "patterns": ["I have a fever", "My head hurts", "I have a cough", "My body aches", "I have chills", "I can't smell anything"],
              "responses": ["""You can make appointment of tele-consultation service in "HA Go" mobile app.
                               Please read the following document for information about receiving treatment for mild cases of Covid-19. 
                              https://www.ha.org.hk/haho/ho/covid-19/designated_clinic_en.pdf"""]
             },
             {"tag": "severe illness",
              "patterns": ["difficulty breathing", "can't speak", "cannot move", "chest pain"],
              "responses": ["If you are severely ill, dial 999 immediately. An ambulance will be on the way to pick you up."]
             },
             {"tag": "vaccination",
              "patterns": ["get the BioNTech shot", "get the Pfizer vaccine", "Sinovac", "covid vaccine", "covid booster shot"],
              "responses": ["""To find out a centre in your district offering vaccination, please type "Covid Vaccination (district)", where (district) is one of the following: \n'Kwun Tong', 'Sha Tin', 'Wan Chai', 'Central & Western', 'Islands', 'Wong Tai Sin', 'Yuen Long', 'Tuen Mun', 'Tsuen Wan', 'Kowloon City', 'Kwai Tsing', 'North', 'Yau Tsim Mong', 'Tai Po', 'Sham Shui Po', 'Eastern', 'Southern', 'Sai Kung' """]
             },
             {"tag": "disease prevention",
              "patterns": ["Protect myself from covid", "prevent myself from becoming ill", "avoid getting sick"],
              "responses": ["To protect yourself, and others, from getting Covid, remember to wash your hands frequently, and always wear a mask in public areas."]
             },
             {"tag": "exposed covid",
              "patterns": ["infections near me", "recent cases in my area", "covid at my workplace", "covid at restaurants I have eaten at"],
              "responses": ["Remember to check LeaveHomeSafe notifications regularly to find out if you have been exposed to Covid. For information on Covid infections near you, check https://chp-dashboard.geodata.gov.hk/covid-19/en.html, and click on the Confirmed Cases Data button."]
             },
             {"tag": "compulsory testing",
              "patterns": ["government mandated testing", "quarantine building", "Do I have to undergo testing"],
              "responses": ["To find out if you have to undergo mandatory Covid testing, visit https://www.chp.gov.hk/files/pdf/ctn.pdf. If your building is on the list, visit https://www.coronavirus.gov.hk/eng/compulsory-testing.html for information on how to get tested."]
             },
             {"tag": "imported cases",
              "patterns": ["Covid imported today", "Covid originated foreign countries", "cases came from other countries"],
              "responses": [f"There have been {rCaseDict['ImportedCasedAdded']} imported cases of Covid-19 today in Hong Kong."]
             },
             {"tag": "Error",
              "patterns": [" "],
              "responses": ["I don't understand. Please try again."]
             }
]}

In [ ]:
#Import the Library
import json
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import tensorflow as tf
import string
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

In [ ]:
# Lemmatize
lemmatizer = WordNetLemmatizer()
words = []
classes = []
documents = []
for intent in training_set["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        documents.append((tokens, intent['tag']))
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
training = []
output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    patterns = doc[0]
    patterns = [lemmatizer.lemmatize(word.lower()) for word in patterns]
    for word in words:
        bag.append(1) if word in patterns else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype=object)
x_train = np.array(list(training[:, 0]))
y_train = np.array(list(training[:, 1]))

In [ ]:
# Define the model
model = Sequential()
model.add(Dense(128, input_shape=(len(x_train[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(y_train[0]), activation = "softmax"))
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
print(model.summary())

In [ ]:
# Train the model
hist = model.fit(x_train, y_train, epochs=200, verbose=1)

In [ ]:
# Functions for chat

def clean_text(text):
     tokens = nltk.word_tokenize(text)
     tokens = [lemmatizer.lemmatize(word) for word in tokens]
     return tokens

def word_bag(text, vocab):
     tokens = clean_text(text)
     bag = [0]*len(vocab)
     for w in tokens:
          for i,j in enumerate(vocab):
               if j == w:
                    bag[i] = 1
     return np.array(bag)
     
def class_prediction(text,vocab,label):
     text = text.lower()
     bag = word_bag(text, vocab)
     p = model.predict(np.array([bag]))[0]
     t = 0.2
     result = [[a,b] for a,b in enumerate(p) if b>t ]
     result.sort(key = lambda x: x[1], reverse = True)
     list_return = []
     
     for r in result :
          list_return.append(label[r[0]])
     if list_return == []:
       return  ['Error']
     else:
       return list_return


def response(intents_list, intents_json): 
     tag = intents_list[0]
     list_of_intents = intents_json["intents"]
     
     for i in list_of_intents: 
          if i["tag"] == tag:
              final_result = random.choice(i["responses"])
              break

     return final_result


In [ ]:
iscontinue = True
print("""Chatbot: Hi. Nice to meet you. I am a AI chatbot. I will try best my best to help you everything about the covid 19 situation and policy in Hong Kong. What can I help you?""")
while iscontinue:
    message = input("Your message: ")
    if message[:17] == "Nearest Hospital ":
        hosp = message[17:]
        print(f"Chatbot: {nearesthospital(hosp)}")
        print('\n')
        continue
    elif message[:18] == "Covid Vaccination ":
        vacc = message[18:]
        print(f"Chatbot: {nearestvacc(vacc)}")
        print('\n')
        continue
    else:
        intents = class_prediction(message, words, classes)
        final_result = response(intents, training_set)
        print(f"Chatbot: {final_result}")
        print('\n')
        if intents ==["bye"]:
            iscontinue = False
        else:
            continue